# Launch post-run analysis jobs
Code to create a batch script for launching jobs on cori
Sep 1, 2020

In [1]:
import os
import glob
import time
import subprocess as sp
import numpy as np

In [2]:
curr_dir=os.getcwd()
print(curr_dir)

/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/1_train/run_scripts


In [3]:
val_files={'128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy',
          '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/512_square/dataset1_smoothing_single_universe/norm_1_train_val.npy'
}

In [4]:
img_size=128
results_loc='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/{0}square/'.format(img_size)
print(results_loc)

/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/


In [5]:
ls /global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/

20200812_072615_bsize256_8gpurun/
20200812_072639_bsize256_1gpurun/
20200812_122131_bsize64_1gpurun/
20200812_122250_bsize64_8gpurun/
20200831_154646_bsize100_128test/
20200911_083711_bsize64_spec_test_128_nospec/
20200911_084729_bsize64_spec_test_128_withspec/
20200914_220904_bsize64_spec_test_128_withspec/
20201028_095913_bsize64_with_spec/
20201029_164804_bsize64_with_spec/
20201030_072640_bsize64_no_adv/
20201109_075645_bsize256_test_256_with_spec_nomcr/
20201109_140046_bsize256_test_b256_with_spec_nomcr_checkpt/
20201109_170341_bsize256_test_b256_lr_0.002/
20201109_171424_bsize256_test_b256_lr_0.0008/
20201110_094334_bsize256_test_b256_lr_0.002_run2/
20201112_073412_bsize256_test_with_invtransform/
20201112_143907_bsize256_test_with_invtransform-scale4.0/
20201117_075959_bsize256_test_with_invtransform-scale1.0/
20201117_083239_bsize256_test_with_invtransform-scale0.5/
20201130_084039_bsize256_test_with_invtransform-scale0.5/
20201202_200004_bsize256_scale0.5/
20201203_084645_bsiz

In [16]:
ip_folder='20201211_201621_bsize256_scale0.1_deterministic_on'


In [17]:
### Default dictionary
dict_pars={'job_name':'analysis', 'queue':'debug',
           'bins_type':'even',# even on uneven
           'time':'00:30:00'}

dict_pars['val_file']=val_files[str(img_size)]
dict_pars['ip_folder']=results_loc+ip_folder

print(dict_pars)

{'job_name': 'analysis', 'queue': 'debug', 'bins_type': 'even', 'time': '00:30:00', 'val_file': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy', 'ip_folder': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20201211_201621_bsize256_scale0.1_deterministic_on'}


In [18]:
assert os.path.isdir(dict_pars['ip_folder'])

In [19]:
bash_strg='''#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos={queue}
#SBATCH --output=slurm-%x-%j.out
#SBATCH --account=m3363
#SBATCH -C haswell
#SBATCH --time={time}
#SBATCH --job-name={job_name}

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

conda activate v3
code_dir='/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/'

python $code_dir/3a_analysis_pandas.py -f {ip_folder}  -v {val_file} --bins_type {bins_type}
conda deactivate
echo "--end date" `date` `date +%s`
'''.format(**dict_pars)

In [20]:
# bash_strg

In [21]:
### Write bash file
staging_loc='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/'
fname='batch_analysis.sh'
filename=staging_loc+fname
print(filename)
with open (filename,'w') as f:
    f.write(bash_strg)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/batch_analysis.sh


In [22]:
### Move to staging locations in project space:
os.chdir(staging_loc)

In [23]:
%%bash -s "$filename" ## Use python variable in bash
cat $1
chmod +x $1

#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos=debug
#SBATCH --output=slurm-%x-%j.out
#SBATCH --account=m3363
#SBATCH -C haswell
#SBATCH --time=00:30:00
#SBATCH --job-name=analysis

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

conda activate v3
code_dir='/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/'

python $code_dir/3a_analysis_pandas.py -f /global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20201211_201621_bsize256_scale0.1_deterministic_on  -v /global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy --bins_type even
conda deactivate
echo "--end date" `date` `date +%s`


### Submit jobs to cori queue

In [24]:
%%bash -s "$filename" ## Use python variable in bash
sbatch $1

Submitted batch job 37236997
